In [1]:
pip install transformers torch


Note: you may need to restart the kernel to use updated packages.Collecting transformers
     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.4 kB ? eta -:--:--
     ----------------- -------------------- 20.5/44.4 kB 320.0 kB/s eta 0:00:01
     -------------------------------------- 44.4/44.4 kB 310.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.7 MB 1.6 MB/s eta 0:00:07
   ---------------------------------------- 0.1/9.7 MB 777.7 kB/s eta 0:00:13
    --------------------------------------- 0.2/9.7 MB 1.3 MB/s eta 0:00:08
    --------------------------------------- 0.2/9.7 MB 1.2 MB/s eta 0:00:08
   - -------------------------------------- 0.3/9.7 MB 1.4 MB/s eta 0:00:07
   - -------------------------------------- 0.4/9.7 MB 1.5 MB/s eta 0:00:07
   - -------------------------------------- 0.5/9.7 MB 1.6 MB/s eta 0:00:06
   

In [3]:
pip install pandas numpy


In [5]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install tensorflow



     ---------------------------------------- 0.0/48.6 kB ? eta -:--:--
     ------------------------- -------------- 30.7/48.6 kB ? eta -:--:--
     -------------------------------- ----- 41.0/48.6 kB 487.6 kB/s eta 0:00:01
     -------------------------------------- 48.6/48.6 kB 407.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/390.3 MB 1.9 MB/s eta 0:03:24
   ---------------------------------------- 0.1/390.3 MB 919.0 kB/s eta 0:07:05
   ---------------------------------------- 0.1/390.3 MB 901.1 kB/s eta 0:07:13
   ---------------------------------------- 0.2/390.3 MB 1.1 MB/s eta 0:06:04
   ---------------------------------------- 0.2/390.3 MB 1.1 MB/s eta 0:06:04
   ---------------------------------------- 0.3/390.3 MB 1.1 MB/s eta 0:06:07
   ---------------------------------------- 0.3/390.3 MB 1.1 MB/s eta 0:06:09
   ---------------------------------------- 0.4/390.3 MB 1.0 MB/s eta 0:0

In [11]:
pip install openpyxl


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Load the dataset
file_path = "C:\\Users\\bjeev\\Downloads\\AI.xlsx"  # Replace with your file path
data = pd.read_excel(file_path)

# Inspect the dataset
print(data.columns)
print(data.head())


Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
        ID    Name               Role  \
0  harima1   Ishan     Data Scientist   
1  harima2   Rajiv       Data Analyst   
2  harima3  Oliver  Software Engineer   
3  harima4   Aarav    Product Manager   
4  harima5   Sonia  Software Engineer   

                                          Transcript  \
0  Interviewer: Hi Ishan, thanks for applying for...   
1  **Interviewer:** Hi Rajiv, thanks for coming i...   
2  Interviewer: Hi Oliver, thanks for taking the ...   
3  **Interviewer:** Hi Aarav, thank you for comin...   
4  Interviewer: Hi Sonia, thanks for coming in to...   

                                              Resume  decision  \
0  **Ishan**\n**Contact Information:**\n\n* Addre...  rejected   
1  **Rajiv**\n**Contact Information:**\n\n*   Ema...  selected   
2  **Oliver**\n**Software Engineer**\n\n**Contact...  selected   
3  **Aarav**\n*

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")


In [9]:
pip install --upgrade transformers


Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
os.environ['TRANSFORMERS_CACHE'] = 'path_to_custom_cache_dir'

from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')


In [13]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')


In [5]:
# Ensure all column names are stripped of whitespace
data.columns = data.columns.str.strip()

# Encode 'ID', 'Role', and target variable 'decision'
label_encoder = LabelEncoder()
data['ID'] = label_encoder.fit_transform(data['ID'])
data['Role'] = label_encoder.fit_transform(data['Role'])
data['decision'] = label_encoder.fit_transform(data['decision'])  # 1 = selected, 0 = rejected

# Additional features to be scaled
additional_features = ['ID', 'Role']

# Scale additional features
scaler = StandardScaler()
X_additional = scaler.fit_transform(data[additional_features])

# Target variable
y = data['decision']


In [17]:
# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Function to generate BERT embeddings
def generate_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()  # [CLS] token embedding
    return cls_embedding

# Generate embeddings for each text column
data['transcript_embeddings'] = data['Transcript'].apply(lambda x: generate_embeddings(str(x)))
data['resume_embeddings'] = data['Resume'].apply(lambda x: generate_embeddings(str(x)))
data['jd_embeddings'] = data['Job Description'].apply(lambda x: generate_embeddings(str(x)))

# Stack embeddings into matrices
transcript_emb = np.vstack(data['transcript_embeddings'])
resume_emb = np.vstack(data['resume_embeddings'])
jd_emb = np.vstack(data['jd_embeddings'])

# Combine all BERT embeddings
X_bert = np.hstack([transcript_emb, resume_emb, jd_emb])


In [19]:
# Combine all features
X = np.hstack([X_bert, X_additional])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:
# Define ANN model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.1),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),

    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),

    Dense(1, activation='sigmoid')  # For binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate ANN
y_pred_ann = model.predict(X_test).round().flatten()
print("ANN Accuracy:", accuracy_score(y_test, y_pred_ann))


C:\Users\bjeev\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 216ms/step - accuracy: 0.4969 - loss: 0.7945 - val_accuracy: 0.5625 - val_loss: 0.6821
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.7615 - loss: 0.5013 - val_accuracy: 0.5000 - val_loss: 0.6856
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.8083 - loss: 0.4396 - val_accuracy: 0.5625 - val_loss: 0.6741
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.8677 - loss: 0.3799 - val_accuracy: 0.6562 - val_loss: 0.6599
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.9521 - loss: 0.3042 - val_accuracy: 0.7500 - val_loss: 0.6533
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8719 - loss: 0.3283 - val_accuracy: 0.7188 - val_loss: 0.6521
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9104 - loss: 0.2476 - val_accuracy: 0.6875 - val_loss: 0.6497
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9438 - loss: 0.1821 - val_accuracy: 0.6875 - val_loss: 0.641

In [23]:
# Train XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Evaluate XGBoost
y_pred_xgb = xgb_model.predict(X_test)
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))


C:\Users\bjeev\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [08:51:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.875


In [25]:
# Combine predictions using average ensemble
ensemble_preds = (y_pred_ann + y_pred_xgb) / 2
ensemble_preds = (ensemble_preds > 0.5).astype(int)

# Evaluate ensemble
print("Ensemble Accuracy:", accuracy_score(y_test, ensemble_preds))


Ensemble Accuracy: 0.65


In [27]:
from sklearn.metrics import accuracy_score, roc_auc_score

# Predictions from the ANN model
y_test_pred_nn = model.predict(X_test)
y_test_pred_nn = [i[0] for i in y_test_pred_nn]  # Extract probabilities if shape is (N, 1)

# Predictions from the XGBoost model
y_test_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]  # Assuming xgb_model is your trained model

# Combine predictions into a DataFrame
test_df = pd.DataFrame()
test_df['actuals'] = y_test
test_df['xgb_pred'] = y_test_pred_proba_xgb
test_df['nn_pred'] = y_test_pred_nn

# Ensemble predictions by averaging the probabilities
test_df['mean_prob'] = (test_df['xgb_pred'] + test_df['nn_pred']) / 2
test_df['new_pred'] = test_df['mean_prob'].round()  # Round to get binary predictions

# Display the DataFrame
print(test_df)

# Evaluate model performance
accuracy = accuracy_score(test_df['actuals'], test_df['new_pred'])
roc_auc_mean = roc_auc_score(test_df['actuals'], test_df['mean_prob'])
roc_auc_xgb = roc_auc_score(test_df['actuals'], test_df['xgb_pred'])
roc_auc_nn = roc_auc_score(test_df['actuals'], test_df['nn_pred'])

print("Ensemble Accuracy:", accuracy)
print("Ensemble ROC AUC Score:", roc_auc_mean)
print("XGBoost ROC AUC Score:", roc_auc_xgb)
print("Neural Network ROC AUC Score:", roc_auc_nn)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step 
     actuals  xgb_pred   nn_pred  mean_prob  new_pred
95         0  0.036769  0.372842   0.204805       0.0
15         1  0.938927  0.496478   0.717702       1.0
30         0  0.087201  0.340263   0.213732       0.0
158        1  0.996469  0.531981   0.764225       1.0
128        0  0.139781  0.397843   0.268812       0.0
115        1  0.690616  0.497026   0.593821       1.0
69         1  0.442244  0.422782   0.432513       0.0
170        1  0.570450  0.399680   0.485065       0.0
174        1  0.872601  0.407361   0.639981       1.0
45         1  0.178821  0.327039   0.252930       0.0
66         1  0.959825  0.520892   0.740359       1.0
182        1  0.729431  0.454026   0.591728       1.0
165        0  0.617870  0.434922   0.526396       1.0
78         0  0.947380  0.330749   0.639065       1.0
186        1  0.978170  0.561561   0.769865       1.0
177        1  0.991679  0.490655   0.741167       1.0
56         1  0.968878  0.511344   0.740111